In [1]:
import sklearn as sk
import seaborn as sns
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [15]:

df_raw = pd.read_excel('trainDataset.xls')
df_raw = df_raw.drop(['RelapseFreeSurvival (outcome)', 'ID'], axis = 1)
#df_test_data = pd.read_excel('testDatasetExample.xls')


print(df_raw.duplicated().sum())
df_raw = df_raw.replace(999, np.nan)
print(df_raw.isna().sum().sum())
df_raw['TrippleNegative'] = df_raw['TrippleNegative'].fillna(df_raw['TrippleNegative'].mode()[0])
df_raw['ChemoGrade'] = df_raw['ChemoGrade'].fillna(df_raw['ChemoGrade'].mode()[0])
df_raw['Proliferation'] = df_raw['Proliferation'].fillna(df_raw['Proliferation'].mode()[0])
df_raw['HistologyType'] = df_raw['HistologyType'].fillna(df_raw['HistologyType'].mode()[0])
df_raw['LNStatus'] = df_raw['LNStatus'].fillna(df_raw['LNStatus'].mode()[0])
df_raw['TumourStage'] = df_raw['TumourStage'].fillna(df_raw['TumourStage'].mode()[0])
print(df_raw.isna().sum().sum())
df_raw = df_raw.dropna()
print(df_raw.isna().sum().sum())

target = df_raw['pCR (outcome)']
learning = df_raw.drop('pCR (outcome)', axis = 1)
cols = list(learning.columns)
scaler = StandardScaler()
scaler.fit(learning)
scaled = scaler.transform(learning)
data = pd.DataFrame(scaled, columns=cols)

pca = PCA(.95)
pca.fit(data)
reduced = pca.transform(data)
print(reduced.shape)



    
    



0
17
7
0
(394, 22)


In [12]:
df = pd.DataFrame(reduced)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,12.697614,1.405147,0.341560,-3.960538,3.767729,1.481145,1.439936,-0.863738,-1.121662,4.271582,...,1.438378,0.875841,-0.292646,-0.495187,-0.430127,-0.028745,1.842995,-0.664447,0.077828,0.355200
1,-3.375096,-2.047934,-2.108632,-1.855581,-1.357780,0.783361,-1.009864,-0.188497,-1.049532,0.873915,...,1.040171,0.080346,0.035084,-1.283058,0.298207,0.002604,0.041206,1.069186,0.100119,0.384018
2,-3.631305,-2.985427,-2.510951,-1.929722,-0.425844,2.324937,1.665576,0.494400,-0.792429,1.688737,...,1.155135,0.607248,-0.795273,-1.547199,1.747064,-0.571943,1.172482,-0.771719,-0.299028,0.404300
3,-6.140871,-1.496980,-2.278612,-2.695956,-0.391183,0.524289,0.934847,1.606983,-1.205792,2.244832,...,0.368168,0.228079,-0.303931,-0.791594,-0.166981,0.962276,-0.676431,0.399265,0.225899,-0.139931
4,-2.189240,-3.709705,-2.317021,-2.940247,-1.090897,2.087253,-1.161792,-0.064244,0.954050,-0.358753,...,0.364554,-0.237807,0.128110,0.164561,0.635767,-0.184497,-0.476638,0.740324,-0.333941,-0.360372


In [25]:
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import GridSearchCV

X = df 
y = target 

gs_logreg = GridSearchCV(LogisticRegression(solver = 'liblinear'),{
    'penalty': ['l1','l2'] #default = l2
}, cv=5, return_train_score=False, scoring = 'accuracy')

gs_logreg.fit(X,y)



GridSearchCV(cv=5, estimator=LogisticRegression(solver='liblinear'),
             param_grid={'penalty': ['l1', 'l2']}, scoring='accuracy')

In [26]:
dfgs = pd.DataFrame(gs_logreg.cv_results_)
dfgs[['param_penalty', 'mean_test_score']]


,param_penalty,mean_test_score
0,l1,0.753846
1,l2,0.738624


In [27]:
from sklearn.linear_model import LinearRegression

linreg = LinearRegression()

gs_linreg = GridSearchCV(LinearRegression(),{
    'fit_intercept': [False,True], #Default = False
    'n_jobs': [0,1,10,50],
}, cv=10, return_train_score=False, scoring = 'neg_mean_squared_error')
gs_linreg.fit(X,y)


GridSearchCV(cv=10, estimator=LinearRegression(),
             param_grid={'fit_intercept': [False, True],
                         'n_jobs': [0, 1, 10, 50]},
             scoring='neg_mean_squared_error')

In [30]:
dfgs = pd.DataFrame(gs_linreg.cv_results_)
dfgs[['param_fit_intercept','param_n_jobs', 'mean_test_score']]

,param_fit_intercept,param_n_jobs,mean_test_score
0,False,0,-0.336161
1,False,1,-0.336161
2,False,10,-0.336161
3,False,50,-0.336161
4,True,0,-0.170805
5,True,1,-0.170805
6,True,10,-0.170805
7,True,50,-0.170805
